In [ ]:
import logging
import os
import sys
import time
from datetime import datetime
from dotenv import load_dotenv

logging.basicConfig(level=logging.INFO)
logging.getLogger("asyncio").setLevel(logging.CRITICAL)
load_dotenv()

root_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.append(root_path)
from core.data_sources.clob import CLOBDataSource
from core.services.mongodb_client import MongoClient
from core.services.backend_api_client import BackendAPIClient
import research_notebooks.statarb_v2.stat_arb_performance_utils as utils

# Initialize clients
clob = CLOBDataSource()
mongo_uri = (
    f"mongodb://{os.getenv('MONGO_INITDB_ROOT_USERNAME', 'admin')}:"
    f"{os.getenv('MONGO_INITDB_ROOT_PASSWORD', 'admin')}@"
    f"{os.getenv('MONGO_HOST', 'localhost')}:"
    f"{os.getenv('MONGO_PORT', '27017')}/"
)
mongo_client = MongoClient(
    uri=mongo_uri,
    database="quants_lab"
)
connector_name = "binance_perpetual"
CONNECTOR_INSTANCE = clob.get_connector(connector_name)
await CONNECTOR_INSTANCE._update_trading_rules()

# Connect to MongoDB
await mongo_client.connect()

# Stat Arb Deploy Tool Demo

This notebook demonstrates the usage of the Stat Arb Deploy Tool methods.

## Fetch Controller Configs and Candles

In [ ]:
# Parameters
days_to_download = 4
interval = "15m"
last_24h = time.time() - 1.5 * 24 * 60 * 60

# Fetch controller configs
controller_configs_data = await mongo_client.get_controller_config_data(min_timestamp=last_24h)

# Get exchange trading pairs
trading_rules = await clob.get_trading_rules(connector_name)
ex_trading_pairs = [trading_rule.trading_pair for trading_rule in trading_rules.data]

# Get trading pairs
base_trading_pairs = [config["config"]["base_trading_pair"] for config in controller_configs_data]
quote_trading_pairs = [config["config"]["quote_trading_pair"] for config in controller_configs_data]
trading_pairs = [trading_pair for trading_pair in list(set(base_trading_pairs + quote_trading_pairs)) if trading_pair in ex_trading_pairs]

rate_limit_config = {
    "okx_perpetual": {
        "batch_size": 3,
        "sleep_time": 10
    },
    "binance_perpetual": {
        "batch_size": 60,
        "sleep_time": 10,
    },
}

# Fetch candles
candles_list = await clob.get_candles_batch_last_days(
    connector_name,
    trading_pairs,
    interval=interval,
    days=days_to_download,
    batch_size=rate_limit_config[connector_name]["batch_size"],
    sleep_time=rate_limit_config[connector_name]["sleep_time"]
)
candles_dict = {candle.trading_pair: candle.data for candle in candles_list}

## Apply Filters to Configs

In [ ]:
# Filter parameters
filter_params = {
    "max_base_step": 0.001,
    "max_quote_step": 0.001,
    "min_grid_range_ratio": 0.5,
    "max_grid_range_ratio": 2.0,
    "max_entry_price_distance": 0.5
}

# Apply filters to configs
filtered_configs = []
for config_data in controller_configs_data:
    if config_data["config"]["base_trading_pair"] not in ex_trading_pairs:
        continue
    if config_data["config"]["quote_trading_pair"] not in ex_trading_pairs:
        continue
    config = config_data["config"]
    extra_info = config_data["extra_info"]
    base_candles = candles_dict[config["base_trading_pair"]]
    quote_candles = candles_dict[config["quote_trading_pair"]]
    
    try:
        meets_condition = await utils.apply_filters(
            connector_instance=CONNECTOR_INSTANCE,
            config=config,
            base_candles=base_candles,
            quote_candles=quote_candles,
            **filter_params
        )
        if meets_condition:
            filtered_configs.append({
                "config": config,
                "base_candles": base_candles,
                "quote_candles": quote_candles,
                "extra_info": extra_info
            })
    except Exception as e:
        print(f"Error processing config: {e}")

print(f"Found {len(filtered_configs)} configs that meet the criteria")

## Visualize Filtered Configs

In [ ]:
import os

top_configs = []

# Iterate over filtered configs and create figures
for i, config_data in enumerate(filtered_configs):
    # Generate the base figure
    fig = await utils.create_coint_figure(
        connector_instance=CONNECTOR_INSTANCE,
        controller_config=config_data["config"],
        base_candles=config_data["base_candles"],
        quote_candles=config_data["quote_candles"],
        extra_info=config_data["extra_info"],
        plot_prices=False
    )

    # Extract details
    base_pair = config_data['config']['base_trading_pair']
    quote_pair = config_data['config']['quote_trading_pair']
    coint_value = config_data['extra_info']['coint_value']
    rate_diff = config_data['extra_info']['rate_difference']

    # Prepare annotation text
    info_text = (f"Config N°{i}<br>"
                 f"Base: {base_pair}<br>"
                 f"Quote: {quote_pair}<br>"
                 f"Coint Value: {coint_value:.3f}<br>"
                 f"Rate Diff: {rate_diff:.5f}%")

    fig.write_image(os.path.join("img", f"config_{i:03d}.jpg"), format="jpg", scale=3)

In [ ]:
selected_index = 3

selected_config_data = filtered_configs[selected_index]

config_to_deploy = selected_config_data["config"].copy()
extra_info = selected_config_data["extra_info"].copy()
base_trading_pair = config_to_deploy["base_trading_pair"]
quote_trading_pair = config_to_deploy["quote_trading_pair"]

if connector_name in ["okx_perpetual"]:
    min_notionals_dict = {trading_rule.trading_pair: float(trading_rule.min_base_amount_increment) * candles_dict[trading_rule.trading_pair].close.iloc[-1] for trading_rule in trading_rules.data if candles_dict.get(trading_rule.trading_pair) is not None}
else:
    min_notionals_dict = {trading_rule.trading_pair: trading_rule.min_notional_size for trading_rule in trading_rules.data}

config_to_deploy

In [ ]:
total_amount_quote = 1000.0
min_spread_between_orders = 0.0004
base_min_order_amount_quote = float(min_notionals_dict[base_trading_pair]) * 1.5
quote_min_order_amount_quote = float(min_notionals_dict[quote_trading_pair]) * 1.5
leverage = 50
time_limit = 259200
stop_loss = 0.1
trailing_delta = 0.005
take_profit = 0.008
activation_price = 0.03

now = datetime.now()
year, iso_week, _ = now.isocalendar()
formatted = f"{year}||isoweek{iso_week}"
controller_id = f"{connector_name}||{config_to_deploy['base_trading_pair']}||{config_to_deploy['quote_trading_pair']}||{formatted}"
tag = "04"

In [ ]:
min_order_amount_quote = max(base_min_order_amount_quote, quote_min_order_amount_quote)
config_to_deploy["id"] = f"{controller_id}_{tag}"
config_to_deploy["total_amount_quote"] = total_amount_quote
config_to_deploy["coerce_tp_to_step"] = True
config_to_deploy["grid_config_base"]["min_order_amount_quote"] = min_order_amount_quote
config_to_deploy["grid_config_quote"]["min_order_amount_quote"] = min_order_amount_quote
config_to_deploy["leverage"] = leverage
config_to_deploy["connector_name"] = "binance_perpetual"  # TODO: restore
config_to_deploy["min_spread_between_orders"] = min_spread_between_orders / 100
config_to_deploy["triple_barrier_config"] = {
    'stop_loss': stop_loss,
    'take_profit': take_profit,
    'time_limit': time_limit,
    'trailing_stop': {'activation_price': activation_price, 'trailing_delta': trailing_delta}
}

In [ ]:
config_to_deploy

In [ ]:
base_candles = candles_dict[base_trading_pair]
quote_candles = candles_dict[quote_trading_pair]

# Create detailed figure
detailed_fig = await utils.create_coint_figure(CONNECTOR_INSTANCE, config_to_deploy, base_candles, quote_candles, extra_info, plot_prices=True)
detailed_fig.update_layout(height=800)

In [ ]:
backend_api_client = BackendAPIClient(host=os.getenv("BACKEND_API_SERVER", "localhost"))

top_configs = []
msg = await backend_api_client.add_controller_config(config_to_deploy)
print(msg)
top_configs.append(config_to_deploy)

In [ ]:
top_configs

In [ ]:
await backend_api_client.deploy_script_with_controllers(bot_name=f"{connector_name}-ts-{iso_week}-{tag}",
                                                        controller_configs=[config["id"] + ".yml" for config in top_configs],
                                                        script_name="v2_with_controllers.py",
                                                        image_name="hummingbot/hummingbot:latest",
                                                        credentials="binance-ts-drupman",
                                                        time_to_cash_out=None,
                                                        max_global_drawdown=None,
                                                        max_controller_drawdown=None)

In [ ]:
# TODO: live performance
# TODO: archive